In [1]:
import pandas as pd
from collections import Counter 
from helper import load_data,pipeline, unique_drugs, generate_drugs_df
import re

%pylab inline 
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [29]:
trials = load_data()
c = unique_drugs(trials)
df, exceptions = generate_drugs_df(c, trials)

got 512 results and had 0 exceptions
probs are[22.66, 29.31, 67.65]
